In [15]:
# !pip install git+https://github.com/NVIDIA-Merlin/models.git

# Environment Setup

Push a Nvidia repo to your artifact repository - (create a new repo if not available)

### Further instructions on how to build a Merlin container can be found [here](https://github.com/jswortz/merlin-gcp)

Guide
1. In an notebook instance run from a terminal `git clone https://github.com/jswortz/merlin-gcp`
2. Run the notebook - this will build a merlin tensorflow image. You can change the base images from NVidia if you choose - this was tested from the version in the repo
3. This will build a container image and save it to artifact registry repository. You may need to enable the artifact repository service
4. Create a new custom workbench by clicking "New Notebook" > "Customize"

<img src="img/custom_workbench.png" width="500"/>

5. To get to the screen for artifact repository. First select "Advanced" then name, select region, then go to "Environment"

<img src="img/workbench-environment.png" width="500"/>

6. Select the your repo and respective image

<img src="img/image.png" width="500"/>

7. Finally, if you already followed the instructions in notebook 02 setup - make sure the new workbench network is in the VPC peered network used for matching engine

# Training with Merlin

## Spotify example

Our goal is to enable the architecture below with Vertex AI and the Merlin Models framework

![](img/arch.png)

## Goals

The goal of this notebook is to create a simple two tower model with Nvtabular and Merlin Models

1. Export data to Parquet for use in Nvtabular
2. Create a NVtabular workflow and fit it to the data
3. Save the fitted workflow to GCS (contains all dictionary lookups)
4. Transform the data into GPU-optimized features from the NVtabular workflow and save to gcs
5. Load the data again as a Merlin dataset using the workflow
6. Create a two tower model in a few lines of code
7. Train a two tower model
8. Save the model (for deployment in notebook 03)
9. Export the embeddings and save to gcs (for deployment to matching engine)

*Based heavily on [this](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) NVIDIA resource


Data is loaded from BQ - to parquet using the job configuration below


Info on the data

* track_uri_seed counts: 2249561
* artist_uri_seed counts: 294110
* album_uri_seed counts: 730377

# Export Data From BigQuery to Parquet on GCS

In [1]:
PROJECT = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
# path = 'gs://two-tower-models' #TODO change to your model directory
BUCKET = 'gs://spotify-beam-v3'
DATASET_ID = 'mdp_eda_test'
TABLE = 'train_flatten_last_5'
TABLE_VALIDATION = 'train_flatten_valid_last_5'

from google.cloud import bigquery
client = bigquery.Client()

def export_bq_to_parquet(PROJECT, DATASET_ID, TABLE, LOCATION, prefix):
    destination_uri = f"{BUCKET}/{prefix}_data_parquet/*.snappy.parquet"
    dataset_ref = bigquery.DatasetReference(PROJECT, DATASET_ID)
    table_ref = dataset_ref.table(TABLE)
    job_config = bigquery.job.ExtractJobConfig()
    job_config.destination_format = bigquery.DestinationFormat.PARQUET
    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        job_config=job_config,
        # Location must match that of the source table.
        location=LOCATION,
    )  # API request
    return(extract_job.result())  # Waits for job to complete.

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


#### Export the data from Bigquery to Parquet

*Run only once* Uncomment to run

In [2]:
# # training
# export_bq_to_parquet(PROJECT, DATASET_ID, TABLE, LOCATION, prefix='train')

# # # validation
# export_bq_to_parquet(PROJECT, DATASET_ID, TABLE_VALIDATION, LOCATION, prefix='valid')

# Create a NVTabular W orkflow To Preprocess The Data

In [3]:
import os
import nvtabular as nvt
from time import time
import pandas as pd
# disable INFO and DEBUG logging everywhere
import logging
import time

logging.disable(logging.WARNING)

from nvtabular.ops import (
    Categorify,
    TagAsUserID,
    TagAsItemID,
    TagAsItemFeatures,
    TagAsUserFeatures,
    AddMetadata,
    ListSlice
)
import nvtabular.ops as ops

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
import tensorflow as tf

# for running this example on CPU, comment out the line below
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

2022-11-05 17:31:01.288625: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-05 17:31:03.105072: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 17:31:04.430700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20480 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found.

### Create NVTabular Dataset
This will be our pre-processor to hash strings and other preprocessing

In [4]:
train = nvt.Dataset(f"{BUCKET}/train_data_parquet/*.snappy.parquet")
valid = nvt.Dataset(f"{BUCKET}/valid_data_parquet/*.snappy.parquet")

#### Define the NVTabular preprocessing graph

Nvtabular takes a tabular dataset and transforms the data into new Parquet files optimized for training. This is equivalent to running a string lookup in tensorflow, and has similar functionality as adpat when a workflow is used

The graph is defined as a set of nodes and a node is defined as follows:

``` 
NODE_NAME = LIST_OF_FEATURE_NAMES >> nvt.ops.* >> [NEXT_STEP] >> Tag
```

Nvt has many ops - one of the most notable is `Categorify` as it is flexible and converts high cardinality text features to indicies

Tags are used as a means to label the schema in the metadata and serves as a label for merlin models to interpret for consumption (e.g. which features belong to the query tower and candidate tower have different tags). You will see the tags in the schema in examples below

In [5]:
item_id = ["track_uri_can"] >> Categorify(dtype="int32") >> TagAsItemID() 
playlist_id = ["pid"] >> Categorify(dtype="int32") >> TagAsUserID() 


item_features_cat = ['artist_name_can',
        'track_name_can',
        'artist_genres_can',
    ]

item_features_cont = [
        'duration_ms_can',
        'track_pop_can',
        'artist_pop_can',
        'artist_followers_can',
    ]

playlist_features_cat = [
        'description_pl',
        'name',
        'collaborative',
    ]

playlist_features_cont = [
        'duration_ms_seed_pl',
        'n_songs_pl',
        'num_artists_pl',
        'num_albums_pl',
    ]

seq_feats_cat = [
        'artist_name_pl',
        'track_uri_pl',
        'track_name_pl',
        'album_name_pl',
        'artist_genres_pl',
    ]

CAT = playlist_features_cat + item_features_cat
CONT = item_features_cont + playlist_features_cont

item_feature_cat_node = item_features_cat >> nvt.ops.FillMissing()>> Categorify(dtype="int32") >> TagAsItemFeatures()

item_feature_cont_node =  item_features_cont >> nvt.ops.FillMissing() >>  nvt.ops.Normalize() >> TagAsItemFeatures()

playlist_feature_cat_node = playlist_features_cat >> nvt.ops.FillMissing() >> Categorify(dtype="int32") >> TagAsUserFeatures() 

playlist_feature_cont_node = playlist_features_cont >> nvt.ops.FillMissing() >>  nvt.ops.Normalize() >> TagAsUserFeatures()

playlist_feature_cat_seq_node = seq_feats_cat >> nvt.ops.FillMissing() >> Categorify(dtype="int32") >> TagAsUserFeatures() 

### Define a workflow
This is an easy step by adding all nodes that need to run in parallel. A graphviz visualization of the graph is available below

In [6]:
# define a workflow
output = playlist_id + item_id \
+ item_feature_cat_node \
+ item_feature_cont_node \
f+ playlist_feature_cat_node \
+ playlist_feature_cont_node \
+ playlist_feature_cat_seq_node 


workflow = nvt.Workflow(output)

In [7]:
output.graph

### Before we transform the data, let's look at an original record
This loads the data into a dask pandas dataframe. `.to_ddf()` converts the dataset into a dask df

In [8]:
# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", None)

valid.to_ddf().head(1)

,name,collaborative,pid,modified_at,pid_pos_id,pos_can,artist_name_can,track_uri_can,album_uri_can,track_name_can,...,pos_pl,artist_name_pl,track_uri_pl,track_name_pl,duration_ms_songs_pl,album_name_pl,artist_pop_pl,artists_followers_pl,track_pop_pl,artist_genres_pl
0,{november '15},false,914400,1452816000,914400-13,13,Tossing Copper,spotify:track:2BQSlFrFfxHjv2O738IzRm,spotify:album:7Bc9G6iDUH9u6pYj6Q5jr7,Edge of Eden,...,"[8, 9, 10, 11, 12]","[Tossing Copper, Tossing Copper, Tossing Copper, Tossing Copper, Tossing Copper]","[spotify:track:4w3eTXloQVxhvc2U3KuuOb, spotify:track:5xdYaI97V4m7pypIhfpkoO, spotify:track:0UDpAmtvdeWJzowyiaE9tI, spotify:track:1sZ2RArZ2F86achracJBBw, spotify:track:7BO96T4UFKUbK96XujhaB2]","[Hello Darling, Silhouettes & Sand, The Bridegroom, Ships of Cortez, Dancing in the Dawn]","[256000.0, 221786.0, 248613.0, 197253.0, 176733.0]","[Silhouettes & Sand, Silhouettes & Sand, Silhouettes & Sand, Silhouettes & Sand, Silhouettes & Sand]","[68.0, 68.0, 68.0, 68.0, 68.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[36.0, 33.0, 32.0, 40.0, 61.0]","['alt z', 'indiecoustica', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'alt z', 'indiecoustica', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'alt z', 'indiecoustica', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'alt z', 'indiecoustica', 'neo mellow', 'pop', 'pop rock', 'post-teen pop', 'alt z', 'indiecoustica', 'neo mellow', 'pop', 'pop rock', 'post-teen pop']"


### Now set the output buckets for the validation and training

In [9]:
output_path = os.path.join(BUCKET, "merlin-processed")
output_train_dir = os.path.join(output_path, 'train/')
output_valid_dir = os.path.join(output_path, 'valid/')
output_workflow_dir = os.path.join(output_path, 'workflow/')


print(f"Train data dir: {output_train_dir}\nValid data dir: {output_valid_dir}")

Train data dir: gs://spotify-beam-v3/merlin-processed/train/
Valid data dir: gs://spotify-beam-v3/merlin-processed/valid/


## Load and preprocess training data back to Parquet GCS
helpful resources https://cloud.google.com/compute/docs/gpus/monitor-gpus

## Also, be sure to install `nvtop` the install script is in `utils/install_nvtop.sh`

## Fit Transform is called on the workflow to pass over the data
The categorical and continuous features must be specified. In this example shuffling is enabled so we only shuflle one time. You can size your output and experiment with number of output files - depending on your use case think of your GPU memory available with respective file sizes

## ** This can take some time (about an hour) but is a one-time save to gcs **

In [10]:
import time

In [13]:
%%time

time_preproc = 0
time_preproc_start = time.time()

workflow.fit_transform(train).to_parquet(output_path=output_train_dir, 
                                         shuffle=nvt.io.Shuffle.PER_PARTITION,
                                         cats=CAT,
                                         conts=CONT, 
                                         output_files = 100
                                         ) #preserve_files=True keeps the original file sharding


time_preproc += time.time()-time_preproc_start

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


CPU times: user 24min 3s, sys: 10min 30s, total: 34min 34s
Wall time: 49min 18s


### Saving to GCS is simple with GCS Fuse, which is already included in the if you built an image from [here](https://github.com/jswortz/merlin-gcp)

In [14]:
#save the workflow to disk (GCS here via ) after it has been fit
workflow.save(os.path.join(output_workflow_dir,'2t-spotify-workflow'))

#locally for demo too
workflow.save('2t-spotify-workflow')

## Transform validation data using the same workflow

This works similarly to a sklearn object where you can fit then transform after

In [15]:
%%time

time_preproc_start = time.time()
workflow.transform(valid).to_parquet(output_path=output_valid_dir, 
                                         shuffle=nvt.io.Shuffle.PER_PARTITION,
                                         cats=CAT,
                                         conts=CONT,
                                         output_files=10
                                         )


time_preproc += time.time()-time_preproc_start

CPU times: user 7.25 s, sys: 3.38 s, total: 10.6 s
Wall time: 16 s


In [16]:
workflow.input_schema

,name,tags,dtype,is_list,is_ragged
0,pid,(),int64,False,False
1,track_uri_can,(),object,False,False
2,artist_name_can,(),object,False,False
3,track_name_can,(),object,False,False
4,artist_genres_can,(),object,False,False
5,duration_ms_can,(),float64,False,False
6,track_pop_can,(),float64,False,False
7,artist_pop_can,(),float64,False,False
8,artist_followers_can,(),float64,False,False
9,description_pl,(),object,False,False


### Examine some of the output data from nvtabular

The data is presented as a Dask Pandas DF - so the interface is very familiar if you know Pandas

Now the data has encoded the values of the strings with categorify, filled missing values, and normalized continuous variables. Now when you look at categorical data like `track_name_pl`, you will see the data represented as an integer, ready to quickly load to an embedding layer, as opposed to hashing each Epoch like when including stringlookups or hash functions in a tfrs model. This ensures the unven profile caused by cpu/gpu bottlnecks during lookup functions are processed just once and efficiently.

#### Example of NVTabular Gpu Consumption Pattern
A100 a1-high-gpu
![Example of Nvtop](img/nvtop.png)

Note that the lookup tables for the various categorified data can be found in the `categories/` folder of the workflow. You also will notice a link in the data schema (more on that below) to the files by each field - this is how you can do reverse lookups later

In [17]:
nvtdata = pd.read_parquet(output_train_dir+'part_0.parquet')
nvtdata['track_name_pl'].head(3) 

0    [11535, 7518, 11313, 42462, 151481]
1               [613, 60, 83, 4161, 252]
2            [4637, 1735, 146, 84, 1886]
Name: track_name_pl, dtype: object

In [18]:
nvtdata = pd.read_parquet(output_train_dir+'part_0.parquet')
nvtdata['artist_name_can'].head(10)

0    2247
1       1
2      12
3    6271
4      54
5      18
6     212
7       3
8    2195
9    8864
Name: artist_name_can, dtype: int32

### Total rows for `part_0.parquet`
Total rows x number of partitions (`output_files` in `workflow.to_parquet()`)

In [19]:
len(nvtdata)

608090

#### Quick view of the embedding dimensions
`(INPUT_CARDINALITY, EMB_DIM)`

In [20]:
embeddings = ops.get_embedding_sizes(workflow)
embeddings

({'pid': (984638, 512),
  'track_uri_can': (2183658, 512),
  'artist_name_can': (278452, 512),
  'track_name_can': (1435485, 512),
  'artist_genres_can': (38183, 512),
  'description_pl': (17909, 385),
  'name': (92266, 512),
  'collaborative': (3, 16)},
 {'artist_name_pl': (283353, 512),
  'track_uri_pl': (2231210, 512),
  'track_name_pl': (1464629, 512),
  'album_name_pl': (563481, 512),
  'artist_genres_pl': (38373, 512)})

In [21]:
print([embeddings[0][x] for x in item_feature_cat_node.output_columns.names])

[(278452, 512), (1435485, 512), (38183, 512)]


In [22]:
playlist_feature_cont_node.output_columns.names

['duration_ms_seed_pl', 'n_songs_pl', 'num_artists_pl', 'num_albums_pl']

# Load the processed data into a Merlin Dataset and Train a Two Tower Model

Now that data inspection is over, the workflow is saved and data is processed to the `output_path`

The above only needs to be run once as well - the workflow and transformed files can be directly loaded from gcs from now on

### Get the embedding dimensions from the workflow

In [10]:
# load back the workflow and schema
# spotify-builtin-2t/merlin-processed/workflow/2t-spotify-workflow
workflow = nvt.Workflow.load(os.path.join(BUCKET, "merlin-processed/workflow/2t-spotify-workflow"))
schema = workflow.output_schema
embeddings = ops.get_embedding_sizes(workflow)
embeddings

({'pid': (984638, 512),
  'track_uri_can': (2183658, 512),
  'artist_name_can': (278452, 512),
  'track_name_can': (1435485, 512),
  'artist_genres_can': (38183, 512),
  'description_pl': (17909, 385),
  'name': (92266, 512),
  'collaborative': (3, 16)},
 {'artist_name_pl': (283353, 512),
  'track_uri_pl': (2231210, 512),
  'track_name_pl': (1464629, 512),
  'album_name_pl': (563481, 512),
  'artist_genres_pl': (38373, 512)})

In [11]:
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset as MerlinDataset

train = MerlinDataset(os.path.join(output_train_dir, "*.parquet"), schema=schema, part_size="1GB")
valid = MerlinDataset(os.path.join(output_valid_dir, "*.parquet"), schema=schema, part_size="1GB")

Direct quote from [here](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)

Other info on modeling setup

### Negative sampling
Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones. In Merlin Models we provide some scaleable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model
Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling here and here), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

- Other model types can be found on the [Merlin Models API Documentation Page](https://nvidia-merlin.github.io/models/main/api.html#tensorflow-models)

### Vertex Experiments

We will organize all of our runs with Merlin in one experiment id `spotify-merlin-train-full-data-v1`. Under this experiment, we will have multiple runs that will be tracked in the experiment interface as well as Tensorboard

In [12]:
# wone_shot_cmd use this to track data in tensorboard
from google.cloud import aiplatform as vertex_ai

TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/7336372589079560192' #fqn - project number then tensorboard id
EXPERIMENT_NAME = f'spotify-merlin-train-full-data-v7-1'
RUN_NAME = f'run-{EXPERIMENT_NAME}-{time.strftime("%Y%m%d-%H%M%S")}'#be sure to think about run and experiment naming strategies so names don't collide
logs_dir = f'{output_path}/tb_logs/{EXPERIMENT_NAME}'

#helper function to get the tb-uploader command

def get_upload_logs_to_manged_tb_command(ttl_hrs, oneshot="false"):
    """
    Run this and copy/paste the command into terminal to have 
    upload the tensorboard logs from this machine to the managed tb instance
    Note that the log dir is at the granularity of the run to help select the proper
    timestamped run in Tensorboard
    You can also run this in one-shot mode after training is done 
    to upload all tb objects at once
    """
    return(f"""tb-gcp-uploader --tensorboard_resource_name={TB_RESOURCE_NAME} \
      --logdir={logs_dir} \
      --experiment_name={EXPERIMENT_NAME} \
      --one_shot={oneshot} \
      --event_file_inactive_secs={60*60*ttl_hrs}""")

vertex_ai.init(experiment=EXPERIMENT_NAME)
    

# we are going to ecapsulate this one-shot log uploader via a custom callback:

class UploadTBLogsBatchEnd(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        os.system(get_upload_logs_to_manged_tb_command(ttl_hrs = 5, oneshot="true"))
    
layers = [512, 256, 128]
LR = .001
BS = 4096
epoch = 120


tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=logs_dir,
        histogram_freq=0, 
        write_graph=True, 
        profile_batch=(20,50) #run profiler on steps 20-40 - enable this line if you want to run profiler from the utils/ notebook
    )


2022-11-05 17:31:13.437652: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-05 17:31:13.437690: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-05 17:31:13.437926: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-05 17:31:13.741270: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-05 17:31:13.741437: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed


In [13]:
from merlin.models.tf.outputs.base import DotProduct, MetricsFn, ModelOutput

user_schema = schema.select_by_tag(Tags.USER)
user_inputs = mm.InputBlockV2(user_schema)

query = mm.Encoder(user_inputs, mm.MLPBlock(layers))

item_schema = schema.select_by_tag(Tags.ITEM)
item_inputs = mm.InputBlockV2(
        item_schema,)
candidate = mm.Encoder(item_inputs, mm.MLPBlock(layers))

model = mm.TwoTowerModelV2(query, candidate)

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

* **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

* **Recall@10:** Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

In [1]:
# model.set_retrieval_candidates_for_evaluation(train)
opt = tf.keras.optimizers.Adagrad(learning_rate=LR)
model.compile(optimizer=opt, run_eagerly=False, 
              metrics=[mm.RecallAt(1), 
                       mm.RecallAt(10), 
                       mm.NDCGAt(10)],
             )

# one_shot_cmd = get_upload_logs_to_manged_tb_command(oneshot="true")
# ! $one_shot_cmd

model.fit(train, 
          validation_data=valid, 
          batch_size=BS, 
          epochs=epoch,
          # steps_per_epoch=20, 
          callbacks=[tensorboard_callback, 
                     UploadTBLogsBatchEnd()],)

#get the metrics for the experiment run
history_keys = model.history.history.keys()

#upload everything to  tb only do this if you were not streaming with the above command


vertex_ai.start_run(RUN_NAME, tensorboard=TB_RESOURCE_NAME)

vertex_ai.log_params({"layers": str(layers), 
                      "learning_rate": LR,
                        "num_epochs": epoch,
                        "batch_size": BS,
                     })

metrics_dict = {}
_ = [metrics_dict.update({key: model.history.history[key][-1]}) for key in history_keys]
metrics_dict
vertex_ai.log_metrics(metrics_dict)
vertex_ai.end_run() #end collecting metrics and the run is complete

- The One-shot command above will download your metrics to tensorboard - and will be avialble in the Experiments tab of Vertex
 
#### Ideally you should be seeing GPU utilization like this - play with batch size to target ~ 70-80% utilization
![](img/merlin-training-nvtop.png)

### Plotting performance of loss and recall

In [18]:
import matplotlib.pyplot as plt

def plot_performance(x, y1, y2, label_1, label_2, label_y):
    plt.plot(x, y1, label = label_1)
    plt.plot(x, y2, label = label_2)
    plt.xlabel('Epoch')
    plt.ylabel(label_y)
    plt.title(label_y)
    plt.legend()
    plt.show()

    
plot_performance(x=range(epoch), 
                 y1=model.history.history['loss'], 
                 y2=model.history.history['val_loss'], 
                 label_1='train-loss', 
                 label_2='val-los', 
                 label_y='loss')

<Figure size 640x480 with 1 Axes>

#### Looking at the recall of predicting the next song by epoch

In [19]:
plot_performance(x=range(epoch), 
                 y1=model.history.history['recall_at_1'], 
                 y2=model.history.history['val_recall_at_1'], 
                 label_1='recall_at_1', 
                 label_2='recall_at_1_val', 
                 label_y='recall_at_1')

#### Save the Query Model

In [1]:
query_tower = model.query_encoder
query_tower.save(os.path.join(output_path, 'query_model_merlin_v2'))


### Save embeddings to GCS for each song - this will become our Matching Engine index

In [24]:
from merlin.models.utils.dataset import unique_rows_by_features

item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ID)
)

item_features_val = (
    unique_rows_by_features(valid, Tags.ITEM, Tags.ID)
)

In [25]:
item_embs = model.candidate_embeddings(
    item_features, index=item_features.schema['track_uri_can'], batch_size=10000
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [26]:
item_embs_val = model.candidate_embeddings(
    item_features_val, index=item_features_val.schema['track_uri_can'], batch_size=10000
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [27]:
# Quick look at the data
item_emb_pd = item_embs_val.compute().to_pandas()
item_emb_pd

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
track_uri_can,,,,,,,,,,,,,,,,,,,,,
0,1.598981,1.148641,0.000000,0.900171,1.039698,1.268149,1.387079,0.872431,0.0,0.000000,...,1.249992,2.830853,1.806437,0.000000,0.0,0.000000,0.965757,2.482678,0.816776,0.000000
1,1.985312,0.314724,0.075375,0.557990,0.000000,0.000000,0.885789,0.000000,0.0,0.000000,...,0.764083,0.000000,1.200484,0.000000,0.0,0.504657,0.000000,2.577287,1.154398,0.040066
2,3.116338,0.000000,0.000000,1.143776,0.000000,0.000000,1.520092,0.000000,0.0,0.000000,...,0.829669,0.000000,0.856309,0.000000,0.0,0.372846,1.147618,2.834061,0.195504,0.000000
3,1.622658,0.000000,0.000000,0.784024,0.000000,0.000000,0.991706,0.000000,0.0,0.000000,...,0.525229,0.000000,1.279989,0.000000,0.0,0.338904,0.000000,2.650746,0.719306,0.000000
4,2.894939,0.193081,0.272784,0.000000,0.000000,0.000000,1.223720,0.000000,0.0,0.000000,...,0.842285,0.000000,0.610949,0.000000,0.0,0.308182,0.000000,2.598973,0.882622,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2183202,0.798891,0.489036,0.000000,1.514931,0.062603,1.786053,1.154257,0.884001,0.0,0.000000,...,1.442423,0.781222,2.487395,0.174898,0.0,0.000000,1.299228,1.578976,1.499897,0.000000
2183320,0.900120,1.202526,0.000000,0.000000,0.036876,0.000000,0.867916,0.000000,0.0,1.087528,...,0.499517,0.421696,2.439277,0.591112,0.0,0.000000,0.121303,1.005926,0.971267,0.373826
2183342,1.155754,0.778916,0.000000,0.000000,0.103660,0.000000,1.022378,0.420565,0.0,0.000000,...,1.135739,0.951849,2.458950,0.969773,0.0,0.066152,0.090780,1.284856,1.820491,0.000000


In [28]:
%%time
!rm candidate_embeddings.json > /dev/null 
!touch candidate_embeddings.json

item_data = pd.read_parquet(os.path.join(BUCKET, "merlin-processed/workflow/2t-spotify-workflow", 'categories/unique.track_uri_can.parquet'))

lookup_dict = dict(item_data['track_uri_can'])

def format_for_matching_engine(data) -> None:
    cols = [str(i) for i in range(layers[-1])] #ensure we are only pulling 0-EMBEDDING_DIM cols
    emb = [data[col] for col in cols]
    formatted_emb = '{"id":"' + str(data['track_uri_can']) + '","embedding":[' + ",".join(str(x) for x in list(emb)) + ']}'
    with open("candidate_embeddings.json", 'a') as f:
        f.write(formatted_emb)
        f.write("\n")
    f.close()

item_emb_pd = item_embs.compute().to_pandas().fillna(1e-10).reset_index() #filling blanks with an epsilon value
item_emb_pd['track_uri_can'] = item_emb_pd['track_uri_can'].apply(lambda l: lookup_dict[l])
_ = item_emb_pd.apply(format_for_matching_engine, axis=1)

item_emb_pd_val = item_embs_val.compute().to_pandas().fillna(1e-10).reset_index() #filling blanks with an epsilon value
item_emb_pd_val['track_uri_can'] = item_emb_pd_val['track_uri_can'].apply(lambda l: lookup_dict[l])
_ = item_emb_pd_val.apply(format_for_matching_engine, axis=1)


rm: cannot remove 'candidate_embeddings.json': No such file or directory
CPU times: user 23min 42s, sys: 31.5 s, total: 24min 13s
Wall time: 24min 58s


In [29]:
!wc -l candidate_embeddings.json

2334932 candidate_embeddings.json


### BE SURE TO RUN `gsutil config` IN TERMINAL TO LOG IN WITH YOUR CREDS

In [30]:
# !pip install gsutil
import os
candidate_path = os.path.join(BUCKET, 'merlin-processed/embeddings/candidate_embeddings.json')

candidate_path

'gs://spotify-beam-v3/merlin-processed/embeddings/candidate_embeddings.json'

In [32]:
# !head -2 candidate_embeddings.json

In [34]:
!gsutil cp candidate_embeddings.json $candidate_path

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Copying file://candidate_embeddings.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 file

## All set move to the next notebook - your bucket now should look something like this:

![](img/merlin-bucket.png)